In [197]:
#pip3 install torch torchvision torchaudio

In [198]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
from torch.utils.data import Dataset, DataLoader

In [199]:
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dimension, n_heads):
        super(MultiHeadAttention, self).__init__()
        assert model_dimension % n_heads == 0
        self.dimension = model_dimension // n_heads
        self.n_heads = n_heads

        # Linear layers for Q, K, V, and output
        self.q = nn.Linear(model_dimension, model_dimension)
        self.k = nn.Linear(model_dimension, model_dimension)
        self.v = nn.Linear(model_dimension, model_dimension)
        self.o = nn.Linear(model_dimension, model_dimension)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.dimension)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention_weights = torch.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, model_dim = x.size()
        x = x.view(batch_size, seq_length, self.n_heads, self.dimension)
        return x.permute(0, 2, 1, 3)  # (batch_size, n_heads, seq_length, dimension)

    def combine_heads(self, x):
        batch_size, n_heads, seq_length, dimension = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(batch_size, seq_length, n_heads * dimension)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.q(Q))
        K = self.split_heads(self.k(K))
        V = self.split_heads(self.v(V))
        attention = self.scaled_dot_product_attention(Q, K, V, mask)
        combined = self.combine_heads(attention)
        return self.o(combined)

In [201]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, model_dimension, ff_dimension):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(model_dimension, ff_dimension)
        self.fc2 = nn.Linear(ff_dimension, model_dimension)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [202]:
class PositionalEncoding(nn.Module):
    def __init__(self, model_dimension, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, model_dimension)
        position = torch.arange(0, max_seq_length).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, model_dimension, 2).float() * -(math.log(10000.0) / model_dimension))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [203]:
class EncoderLayer(nn.Module):
    def __init__(self, model_dimension, n_heads, ff_dimension, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(model_dimension, n_heads)
        self.feed_forward = PositionWiseFeedForward(model_dimension, ff_dimension)
        self.norm1 = nn.LayerNorm(model_dimension)
        self.norm2 = nn.LayerNorm(model_dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        return self.norm2(x + self.dropout(ff_output))

Now, we will be working on the decoder blocks

In [204]:
class DecoderLayer(nn.Module):
    def __init__(self, model_dimension, n_heads, ff_dimension, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(model_dimension, n_heads)
        self.cross_attn = MultiHeadAttention(model_dimension, n_heads)
        self.feed_forward = PositionWiseFeedForward(model_dimension, ff_dimension)
        self.norm1 = nn.LayerNorm(model_dimension)
        self.norm2 = nn.LayerNorm(model_dimension)
        self.norm3 = nn.LayerNorm(model_dimension)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        cross_attn_output = self.cross_attn(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))
        ff_output = self.feed_forward(x)
        return self.norm3(x + self.dropout(ff_output))

Combining the Encoder and Decoder layers to create the complete Transformer network

In [205]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, model_dimension, n_heads, num_layers, ff_dimension, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, model_dimension)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, model_dimension)
        self.positional_encoding = PositionalEncoding(model_dimension, max_seq_length)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(model_dimension, n_heads, ff_dimension, dropout) for _ in range(num_layers)]
        )
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(model_dimension, n_heads, ff_dimension, dropout) for _ in range(num_layers)]
        )
        self.fc = nn.Linear(model_dimension, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
        seq_length = tgt.size(1)
        nopeak_mask = torch.triu(torch.ones(seq_length, seq_length), diagonal=1).bool().to(tgt.device)
        tgt_mask = tgt_mask & ~nopeak_mask.unsqueeze(0)
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)

        return self.fc(dec_output)

In [206]:
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab, max_seq_length):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src = self.encode(self.src_sentences[idx], self.src_vocab)
        tgt = self.encode(self.tgt_sentences[idx], self.tgt_vocab)
        return torch.tensor(src), torch.tensor(tgt)

    def encode(self, sentence, vocab):
        tokens = [vocab.get(token, vocab["<unk>"]) for token in sentence.split()]
        tokens = tokens[:self.max_seq_length - 1] + [vocab["<eos>"]]
        return tokens + [vocab["<pad>"]] * (self.max_seq_length - len(tokens))

Training the Model

In [207]:

src_vocab = {"<pad>": 0, "<eos>": 1, "<unk>": 2, "How": 3, "are": 4, "you": 5, "?": 6}
tgt_vocab = {"<pad>": 0, "<eos>": 1, "<unk>": 2, "Comment": 3, "allez-vous": 4, "?": 5}

# Example sentences
src_sentences = ["How are you?", "What is your name?"]
tgt_sentences = ["Comment allez-vous?", "Quel est votre nom?"]

transformer = Transformer(
    src_vocab_size=len(src_vocab),
    tgt_vocab_size=len(tgt_vocab),
    model_dimension=512,
    n_heads=8,
    num_layers=6,
    ff_dimension=2048,
    max_seq_length=10,
    dropout=0.1,
)


dataset = TranslationDataset(src_sentences, tgt_sentences, src_vocab, tgt_vocab, max_seq_length=10)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [208]:
import time

In [209]:
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab["<pad>"])
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

#start time
start = time.time()

for epoch in range(10):
    epoch_loss = 0
    for src_batch, tgt_batch in data_loader:
        optimizer.zero_grad()

        # Remove the last token from the target for input and first for target
        src, tgt_input, tgt_output = src_batch, tgt_batch[:, :-1], tgt_batch[:, 1:]

        # Forward pass
        output = transformer(src, tgt_input)
        loss = criterion(output.reshape(-1, len(tgt_vocab)), tgt_output.reshape(-1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(data_loader)}")

#end time
end = time.time()

print(f"Time taken: {end-start} seconds")

Epoch 1, Loss: 1.7579504251480103
Epoch 2, Loss: 1.4884840250015259
Epoch 3, Loss: 0.5105863213539124
Epoch 4, Loss: 1.0654737949371338
Epoch 5, Loss: 0.5923992991447449
Epoch 6, Loss: 0.6513323187828064
Epoch 7, Loss: 0.8769221305847168
Epoch 8, Loss: 0.7369091510772705
Epoch 9, Loss: 0.48680350184440613
Epoch 10, Loss: 0.5094538331031799
Time taken: 1.1457328796386719 seconds
